In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import sklearn.linear_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import sklearn.multioutput
import tensorflow_text as text  # tf registers ops on import
import tensorflow as tf
from tensorflow.keras.regularizers import L2
from tensorflow.keras.metrics import Precision
import tensorflow_hub as hub
import transformers

from retrain_bert import settings
from retrain_bert.preprocessor import load_labels, get_labels_conf

2023-12-29 21:55:31.756313: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-29 21:55:31.756354: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-29 21:55:31.758416: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-29 21:55:31.890006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-29 21:55:33.276059: W tensorflow/compiler/

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available.")
else:
    print("No GPU detected.")

GPU is available.


2023-12-29 21:55:34.637664: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-29 21:55:34.822209: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-29 21:55:34.822440: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
if Path('/code').exists():
    settings.PROJECT_DIR = Path('/code')
    settings.INFERENCE_MODEL_PATH = settings.PROJECT_DIR / "models/bert_finetuned.h5"
    settings.CATEGORIES_PATH = settings.PROJECT_DIR / "data/categories.csv"

In [ ]:
train_data = pd.read_csv(settings.DATA_DIR / "train.csv")
extra_data = pd.read_csv(settings.PROJECT_DIR / "data/")

In [4]:
train_data, val_data = train_test_split(pd.read_csv(settings.PROJECT_DIR / "data/train/train.csv").dropna(subset=["OcrValue"]), random_state=42)
labels = load_labels()

In [ ]:
extra_data = pd.read_csv(settings.PROJECT_DIR / "data/train/")

In [5]:
fuzzy_mappings = pd.read_csv(
    settings.PROJECT_DIR / "data/raw/Fuzzy_Mappings.csv",
    usecols=["OcrValue", "Correct_Desc"]
).dropna(subset=["Correct_Desc", "OcrValue"]
).drop_duplicates(subset=["OcrValue"])
unlabelled = pd.read_csv(settings.PROJECT_DIR / "data/raw/unverified classifications.csv").dropna(subset=["OCR"])
try:
    generated_train_data = pd.read_csv(settings.PROJECT_DIR / "data/train/generated_train.csv").dropna(subset=["OcrValue"])
    unlabelled = unlabelled.query("OCR not in @generated_train_data.OcrValue")
except FileNotFoundError:
    generated_train_data = None

In [6]:
fuzzy_validation = pd.merge(
    val_data,
    fuzzy_mappings,
    left_on="OcrValue",
    right_on="Correct_Desc",
    how="inner",
    suffixes=["_l", "_r"]
).drop(columns=["OcrValue_l", "Correct_Desc"]
).rename(columns={"OcrValue_r": "OcrValue"})

val_data = pd.concat([val_data, fuzzy_validation], ignore_index=True)

In [7]:
fuzzy_train = pd.merge(
    train_data,
    fuzzy_mappings,
    left_on="OcrValue",
    right_on="Correct_Desc",
    how="inner",
    suffixes=["_l", "_r"]
).drop(columns=["OcrValue_l", "Correct_Desc"]
).rename(columns={"OcrValue_r": "OcrValue"})

train_data = pd.concat([train_data, fuzzy_train], ignore_index=True)

In [8]:
unlabelled = unlabelled.query("OCR not in @train_data.OcrValue")
unlabelled = unlabelled.query("OCR not in @val_data.OcrValue")

In [9]:
import unidecode
def clean_ocrs(ocrs: pd.Series):
    ocrs = ocrs.str.replace(r"\s+", " ")
    ocrs = ocrs.str.strip()
    ocrs = ocrs.str.lower()
    ocrs = ocrs.apply(unidecode.unidecode)
    return ocrs

train_data["OcrValue"] = clean_ocrs(train_data["OcrValue"])
val_data["OcrValue"] = clean_ocrs(val_data["OcrValue"])
unlabelled["OCR"] = clean_ocrs(unlabelled["OCR"])

In [10]:
labels_conf = get_labels_conf(labels=labels)

In [11]:
text_inputs = tf.keras.layers.Input(shape=(), dtype=tf.string)
# TODO: reduce seq_length (128 is an overkill)
# https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3
# data_augmentation = ReplaceWithSynonims(ft_model)
#tokenizer_inputs = data_augmentation(text_inputs)
lowercase = tf.strings.lower(text_inputs)
tokenizer_inputs = lowercase
tokenizer = hub.KerasLayer(
   "https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3",
    #arguments={"seq_length": 32}
)
encoder_inputs = tokenizer(tokenizer_inputs)
encoder = hub.KerasLayer(
   "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4",
    trainable=False
)
encoder_outputs = encoder(encoder_inputs)
pooled_output = encoder_outputs["pooled_output"]
pooled_output = tf.keras.layers.GaussianNoise(0.1)(pooled_output)
X = tf.keras.layers.Dense(settings.DEEPEST_LEVEL * settings.DEEPEST_LEVEL, activation="relu")(pooled_output)
X = tf.keras.layers.Dense(settings.DEEPEST_LEVEL * settings.DEEPEST_LEVEL, activation="relu")(X)
X = tf.keras.layers.Dense(settings.DEEPEST_LEVEL * settings.DEEPEST_LEVEL, activation="relu")(X)
X = tf.keras.layers.Dense(settings.DEEPEST_LEVEL * settings.DEEPEST_LEVEL, activation="relu")(X)

sequence_output = encoder_outputs["sequence_output"]
sequence_output = tf.keras.layers.GaussianNoise(0.1)(sequence_output)
sequence_output = tf.keras.layers.Dropout(0.5)(sequence_output)
sequence_output = tf.keras.layers.Conv1D(16, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Conv1D(32, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Conv1D(64, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Conv1D(128, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Conv1D(256, 3, padding="same", kernel_regularizer=L2(1e-5))(sequence_output)
sequence_output = tf.keras.layers.Flatten()(sequence_output)

X = tf.keras.layers.Concatenate()([X, sequence_output])
X = tf.keras.layers.Dropout(.5)(X)

heads = []
for level in range(settings.DEEPEST_LEVEL):
    head = tf.keras.layers.Dense(
        labels_conf[level]["num_classes"],
        activation="softmax",
        name=f"level_{level}",
        kernel_regularizer=L2(1e-5)
    )(X)
    X = tf.keras.layers.Concatenate()([X, head])
    heads.append(head)

model = tf.keras.Model(text_inputs, heads)

2023-12-29 21:57:05.792825: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-29 21:57:05.793165: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-29 21:57:05.793396: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [12]:
def reshape_for_fit(*batch):
    X = batch[0]
    num_classes = len(batch) // 2
    y = batch[1:num_classes + 1]
    sample_weights = batch[num_classes + 1:]
    return X, y, sample_weights

def _calculate_sample_weights(targets):
    sample_weights = [np.ones(len(targets[0])) for _ in range(settings.DEEPEST_LEVEL)]
    for level in range(settings.DEEPEST_LEVEL):
        unknown_label = labels_conf[level]["num_classes"] - 1
        sample_weights[level][targets[level][:, unknown_label] == 1] = 0
    return sample_weights

def _create_dataset(data, targets, sample_weights, shuffle=True):
    target_datasets = [tf.data.Dataset.from_tensor_slices(t).map(lambda x: tf.cast(x, tf.float32)) for t in targets]
    sample_weight_datasets = [tf.data.Dataset.from_tensor_slices(w) for w in sample_weights]

    dataset = tf.data.Dataset.zip(tf.data.Dataset.from_tensor_slices(data), *target_datasets, *sample_weight_datasets)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(data))
    return dataset.prefetch(tf.data.AUTOTUNE)

def encode_label(label_ids, labels_table=labels):
    one_hot = np.zeros(len(labels_table))
    one_hot[label_ids] = 1
    return one_hot

def make_targets(data):
    labels = data.drop("OcrValue", axis="columns")
    targets = []
    for level, col in zip(range(settings.DEEPEST_LEVEL), labels.columns):
        level_targets = np.zeros((len(labels), labels_conf[level]["num_classes"]))
        for i, label in enumerate(labels[col]):
            level_targets[i, label] = 1
        targets.append(level_targets.astype(np.float32))

    return targets

def create_balanced_dataset(data, ignore_unknown=False):
    datasets = []
    lengths = data.groupby("level_1").count()["OcrValue"]
    groups = data.groupby("level_1")
    for _category, group in groups:
        targets = make_targets(group)
        if ignore_unknown:
            sample_weights = _calculate_sample_weights(targets)
        else:
            sample_weights = [np.ones(len(group)) for _ in range(settings.DEEPEST_LEVEL)]
        datasets.append(_create_dataset(group["OcrValue"], targets, sample_weights=sample_weights))
    return tf.data.Dataset.sample_from_datasets(datasets, stop_on_empty_dataset=True)

def create_train_dataset(model, labelled_data, generated_data, unlabelled_data, batch_size):
    verified_dataset = create_balanced_dataset(labelled_data)

    if generated_data is not None:
        generated_dataset = create_balanced_dataset(generated_data)
        train_dataset = verified_dataset.concatenate(generated_dataset)
    else:
        train_dataset = verified_dataset

    if unlabelled_data is not None:
        unlabelled_sample = unlabelled_data.sample(n=len(labelled_data) // 10).values
        unlabelled_targets = model.predict(unlabelled_sample, batch_size=batch_size * 8)
        unlabelled_target_datasets = [tf.data.Dataset.from_tensor_slices(t) for t in unlabelled_targets]
        unlabelled_sample_weight_datasets = [tf.data.Dataset.from_tensor_slices(np.ones(len(unlabelled_sample))) for _ in range(settings.DEEPEST_LEVEL)]
        unlabelled_dataset = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(unlabelled_sample), *unlabelled_target_datasets, *unlabelled_sample_weight_datasets))
        mixed_dataset = tf.data.Dataset.sample_from_datasets([train_dataset, unlabelled_dataset], stop_on_empty_dataset=True)
    else:
        mixed_dataset = train_dataset

    return mixed_dataset.map(reshape_for_fit).shuffle(buffer_size=len(labelled_data)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [16]:
train_data

,OcrValue,level_1,level_2,level_3,level_4,level_5
0,salch.roja sadepor,2,2,13,25,19
1,g-daurada marbu 600,1,0,10,0,19
2,galletas mantequ,1,0,10,4,19
3,rodaballo,2,7,0,25,19
4,dulano selection/jamon c,2,2,10,25,19
...,...,...,...,...,...,...
3571225,huevo sorpresa kinder 30 pr,1,0,3,25,19
3571226,planta de interior mix,4,1,4,1,7
3571227,planta de interior mix a,4,1,4,1,7
3571228,planta de interior hix,4,1,4,1,7


In [13]:
val_labels = make_targets(val_data)

In [14]:
raise InterruptedError("Stop here")

InterruptedError: Stop here

In [15]:
ds = create_train_dataset(model, train_data, None, unlabelled_data=None, batch_size=64)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=[Precision()]
)

model.fit(ds, epochs=1, validation_data=(val_data.OcrValue, val_labels))

for epoch in range(3):
    print(f"Epoch {epoch}")
    model.fit(ds, epochs=1, validation_data=(val_data.OcrValue, val_labels))
model.save(settings.PROJECT_DIR / "models/bert_finetuned.h5")

2023-12-29 22:02:29.134270: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:55: Filling up shuffle buffer (this may take a while): 1169232 of 1443855
2023-12-29 22:02:31.332245: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2023-12-29 22:02:41.332457: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:36: Filling up shuffle buffer (this may take a while): 1216667 of 1625713
2023-12-29 22:02:45.008334: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2023-12-29 22:02:51.036367: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2023-12-29 22:02:53.491977: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-29 22:02:54.164453: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f976c0134d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-

      1/Unknown - 54s 54s/step - loss: 14.4484 - level_0_loss: 1.6838 - level_1_loss: 2.4312 - level_2_loss: 3.4050 - level_3_loss: 3.3662 - level_4_loss: 3.5569 - level_0_precision: 0.0000e+00 - level_1_precision: 0.0000e+00 - level_2_precision: 0.0000e+00 - level_3_precision: 0.0000e+00 - level_4_precision: 0.0000e+00

2023-12-29 22:03:00.402796: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12078310055039020920
2023-12-29 22:03:00.402830: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9877868177061700066
2023-12-29 22:03:00.402844: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12362403411591484024
2023-12-29 22:03:00.402854: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15998968486670868742
2023-12-29 22:03:00.402864: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4468024859571947192
2023-12-29 22:03:00.402874: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13360740432575469414
2023-12-29 22:03:00.402883: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv

KeyboardInterrupt: 

In [ ]:
model.load_weights(settings.PROJECT_DIR / "models/bert_finetuned.keras")

In [ ]:
from datetime import datetime

val_sample = val_data.sample(n=10000)
val_sample_labels = make_targets(val_sample)

for layer in model.layers:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

for epoch in range(5):
    print(f"Epoch {epoch}")
    ds = create_train_dataset(model, train_data, None, unlabelled_data=unlabelled.OCR, batch_size=32)
    model.fit(ds, epochs=1, validation_data=(val_sample.OcrValue, val_sample_labels))
    seconds_since_epoch = (datetime.now() - datetime(1970,1,1)).total_seconds()
    model.save(settings.PROJECT_DIR / f"models/bert_{seconds_since_epoch}.keras")

Epoch 0


ERROR:absl:hub.KerasLayer is trainable but has zero trainable weights.
2023-11-20 15:39:24.795677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


1503/1503 [==============================] - 3927s 3s/step


2023-11-20 16:45:25.463577: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:36: Filling up shuffle buffer (this may take a while): 1085189 of 1682080
2023-11-20 16:45:30.965642: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2023-11-20 16:45:44.135282: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 1127388 of 1781625
2023-11-20 16:45:50.892014: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2023-11-20 16:45:55.433756: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:113: Filling up shuffle buffer (this may take a while): 243903 of 3846790
2023-11-20 16:46:00.830227: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2023-11-20 16:46:01.926877: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f4db40a5190 initialized for platform CUDA (this d

  12562/Unknown - 7572s 596ms/step - loss: 2.7363 - level_0_loss: 0.2368 - level_1_loss: 0.3746 - level_2_loss: 0.6321 - level_3_loss: 0.7326 - level_4_loss: 0.7549 - level_0_precision: 0.9902 - level_1_precision: 0.9845 - level_2_precision: 0.9791 - level_3_precision: 0.9572 - level_4_precision: 0.9213

...

2023-11-20 23:11:40.013180: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:306: Filling up shuffle buffer (this may take a while): 1137426 of 1682080
2023-11-20 23:11:44.443927: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2023-11-20 23:11:54.444367: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:287: Filling up shuffle buffer (this may take a while): 1255823 of 1781625
2023-11-20 23:11:59.807205: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2023-11-20 23:12:06.969432: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:383: Filling up shuffle buffer (this may take a while): 365468 of 3846790
2023-11-20 23:12:09.741518: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


  12611/Unknown - 7656s 604ms/step - loss: 2.5261 - level_0_loss: 0.2258 - level_1_loss: 0.3622 - level_2_loss: 0.5712 - level_3_loss: 0.6691 - level_4_loss: 0.6924 - level_0_precision: 0.9917 - level_1_precision: 0.9864 - level_2_precision: 0.9815 - level_3_precision: 0.9610 - level_4_precision: 0.9269

2023-11-21 01:19:02.923059: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3243726428165320706
2023-11-21 01:19:02.923091: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3064229816725664484
2023-11-21 01:19:02.923101: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11164665848169906854
2023-11-21 01:19:02.923109: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9688279026045646034
2023-11-21 01:19:02.923118: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16220204542225118254
2023-11-21 01:19:02.923126: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 713505069548910102
2023-11-21 01:19:02.923134: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv it

12611/12611 [==============================] - 7731s 610ms/step - loss: 2.5261 - level_0_loss: 0.2258 - level_1_loss: 0.3622 - level_2_loss: 0.5712 - level_3_loss: 0.6691 - level_4_loss: 0.6924 - level_0_precision: 0.9917 - level_1_precision: 0.9864 - level_2_precision: 0.9815 - level_3_precision: 0.9610 - level_4_precision: 0.9269 - val_loss: 3.7995 - val_level_0_loss: 0.2042 - val_level_1_loss: 0.4194 - val_level_2_loss: 0.7869 - val_level_3_loss: 1.1397 - val_level_4_loss: 1.2438 - val_level_0_precision: 0.9490 - val_level_1_precision: 0.9184 - val_level_2_precision: 0.8828 - val_level_3_precision: 0.7937 - val_level_4_precision: 0.6870
Epoch 3
 748/1503 [=============>................] - ETA: 33:07

KeyboardInterrupt: 